全てのラベルを1に割り振ってしまう、という状態。
過学習しない・量的変数を増やす事を目標にする

仮説：
木のダメージにまつわる本質的な情報を取り出せると精度が向上する。
歩道に生えているか、管理者は何人か、管理している人は専門家かボランティアか
ダメージに関係しそうな木の問題はあるか、木の種類は何か、
行政区分＝木の管理に割かれている予算はどれほどか、といった情報が抽出できると
精度が向上するはず。

これまでに試した分析法
- ["curb_loc","steward","guards","sidewalk","user_type"]をダミー化する(drop_first=False)
- "problems"をダミー化する。その際最初の10行だけを残す

20231213、今日試す分析法
- spc_commonをダミー化。その際最初の15行だけを残す→効果なし
- problems、spc＿commonを量的変数を出現頻度を表す量的変数にする→効果なし

- borocodeをダミー化

# - stewardを量的変数に戻す。1.5, 3.5, 5の3値にする
boro_ct ニューヨーク市のセンサストラクト（小地域統計区域）のコード
cb_num コミュニティボード番号
st_senate 州上院地区番号
st_assem 州議会地区番号
cncldist 市議会の地区番号
→行政区分をダミー化、もしくは頻度の量的変数にする

→ ダミー化は全く効かない。代わりにこれまでダミー化した値すべてをターゲットエンコーディングに切り替える。

Target Encodingの変換 =「目的変数の平均値を特徴量にする」
- ["curb_loc","steward","guards","sidewalk","user_type","borocode"]をターゲットエンコーディングする
→効果なし。他のすべての変数をターゲットエンコーディングしてみる
["nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]をターゲットエンコーディング



木の健康状態（目的変数）
0:Fair（普通） 1:Good（良い） 2:Poor（悪い）
0:良い、1:普通、2:悪い、という値に置き換える

→ミクロF!スコアが比較的高い分類モデルの混合行列を検討する。
LGBMのミクロF1スコアは0.7905929447085315

→比重付き精度(Balanced score)が比較的高い分類モデルの混合行列を検討する。
LGBMの比重付き精度は0.3471486196238671

NearestCentroid　0.49
GaussianNB　0.40
PassiveAggressiveClassifier	0.39
QuadraticDiscriminantAnalysis 0.39
ExtraTreeClassifier	0.38
DecisionTreeClassifier　0.37
LogisticRegression　0.36



全ての変数をターゲットエンコーディングしたところ、ようやく精度が向上した。
一見木とか関係なさそうな 州議会地区番号・市議会の地区番号が実が重要、ということが分かる。

 ["tree_dbh","curb_loc","steward","guards","sidewalk","user_type","problems","spc_common","nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]
y_list = ["health"]

	submit_randomforest_01.csv	0.3108493	木の分析、ターゲットエンコーディング +...修正	2023-12-14 10:08:23
	submit_randomforest_01.csv	0.1165707	木の分析、ランダムフォレスト法による予測修正	2023-12-13 14:33:06
	submit_gnb_01.csv	0.3415307	木の分析、ガウシアンナイーブベイズによる...修正	2023-12-13 14:27:36
	submit_lgbm_02.csv	0.2957859	木の分析、LGBMの改良版修正	2023-12-13 14:13:35
	submit_lgbm_nearcenter01.csv	0.3248569	木の分析、近傍中心法を使った予測修正	2023-12-13 14:06:32
	submit_lgbm_logreg01.csv	0.3045470	木の分析、ロジスティック回帰を使った予測修正	2023-12-13 14:04:06
	submit_lgbm_baseline01.csv	0.2943655	LGBMによるベースライン、6変数のみに...

In [ ]:
精度が高くなる予測モデルの特徴
→ Accuracyが過学習していない(0.79にならない、混合行列を見た際の再現度が高い)、
ミクロF1スコアが高い、

GaussianNB
マクロf1スコア：
0.3803493338446983

PassiveAggressiveClassifier
マクロf1スコア：
0.4168970929151401

ExtraTreeClassifier
マクロf1スコア：
0.37019650218319544

DecisionTreeClassifier
マクロf1スコア：
0.37882986018366616

QuadraticDiscriminantAnalysis
マクロf1スコア：
0.3720410136832782

LinearDiscriminantAnalysis
マクロf1スコア：
0.3581063198509942

BaggingClassifier
マクロf1スコア：
0.3713729527219161

Perceptron
マクロf1スコア：
0.35592729143234053

ExtraTreesClassifier
マクロf1スコア：
0.350071925186423

KNeighborsClassifier	

In [ ]:
LGBMで精度向上 → 最適化時の指標をマクロF1スコアにする
指標をマクロF1にするだけではこれ以上改善できない。
他のアルゴリズムを試したら特徴量エンジニアリングによる精度向上に
移行する

→ラベルが実際１・2の場合どのようなスコアが多いか、という特徴を
　分析する。ダミー化等でそうした指標を狙って値に入れる

In [ ]:
暫定評価が高いモデルでアンサンブル学習する
→ ガウシアンナイーブベイズ

In [ ]:
PassiveAggressiveClassifier
GaussianNB

In [ ]:
ターゲットエンコーディングに加え、
ユニーク数が少ない質的変数のダミー、
problemsの有無を含めたダミーを入れて
lazypredict時の値が向上するか検証する！

In [ ]:
次元圧縮 → うまく特徴量を抽出できていれば
t-SNEで圧縮した時にクラスターが浮かび上がるはず

In [ ]:
t-SNEで圧縮したデータを学習させて精度を検証する
→逆効果。16次元から2次元の圧縮では効果は薄い

In [ ]:
curb_loc, steward, guards, sidewalk, user_typeなど
木の状態に関係する値をダミー化する
problemsのみ、そのブール値をダミー化する

bool_problems

In [ ]:
効果のあった特徴量エンジニアリング・予測モデル

・ダミー化 → 効果なし
・ターゲットエンコーディング → 質的変数が多い場合に有効
　リーク情報を含めると精度が向上する
・木の種類→種類からより分析に有効な値を抽出
 
・LazyPredict → F1の高いモデルが有効

・木ダメージ=特徴量を抽出すればより精度を高めることができる
・時系列データ＝木ダメージと関係あるかも？
・"Problems" - どう処理するか？→問題が多いほどダメージがある

0：良い、1：普通、2：悪い

In [ ]:
GaussianNB

PassiveAggressiveClassifier

LinearDiscriminantAnalysis

ExtraTreesClassifier

BaggingClassifier

ExtraTreeClassifier

In [ ]:
20231215
・変数全てをターゲットエンコーディング
["curb_loc","steward","guards","sidewalk","user_type","problems","spc_common","nta","borocode","boro_ct", "zip_city", "cb_num", "st_senate", "st_assem", "cncldist"]
・少数の質的変数をダミー化
["curb_loc","steward","guards","sidewalk","user_type","borocode"]
・標準化
・特徴量選択 → 寄与率の低いノイズとなる変数を弾く

In [ ]:
改善点
ダミー化する変数に行政区分の情報を加える
borocode


In [ ]:
20231216
データの不均衡度を検証、アンダーサンプリング・オーバーサンプリングで過学習対策を加える
マッピング後の0の割合を半分に、それで精度が向上したら1/4に削減する

 pip install -U imbalanced-learn
 →アンダーサンプリング用のライブラリ
 
 
0の割合：
78.81805444355484 %
半分：39.409
 1/4：19.705
 
1の割合：
17.689151321056844 %

2の割合：
3.4927942353883106 %

→割合を30,15,5にする

4 / 50
16 / 50
30 / 50

sum - 19984
multiple

→アンダーサンプリングは逆効果。
 2の値をオーバーサンプリングする事を目指す！
→等倍へのオーバーサンプリングは逆効果。
次は1・2のみを2倍に増やして検証する
 
 

In [ ]:
	Accuracy	Balanced Accuracy    ROC AUC　F1 Score Time Taken

ExtraTreesClassifier	0.89	0.89	None	0.89	5.81
model_etc
比重付き精度：
0.8864587541705916
マクロf1スコア：
0.8860502719302964

RandomForestClassifier	0.88	0.88	None	0.88	10.44
model_rf
比重付き精度：
0.882226755930342
マクロf1スコア：
0.8815924401494835

XGBClassifier	        0.86	0.86	None	0.86	6.64
LGBMClassifier	        0.84	0.84	None	0.83	1.20
BaggingClassifier	    0.82	0.82	None	0.82	6.08
KNeighborsClassifier	0.78	0.78	None	0.78	0.77
ExtraTreeClassifier	    0.76	0.76	None	0.76	0.14
DecisionTreeClassifier	0.73	0.73	None	0.73	0.67

In [ ]:
	                Accuracy BAccuracy ROC AUC　F1Score Time Taken

Model					
LabelPropagation	    0.71	0.61	None	0.71	98.58
LabelSpreading	        0.71	0.61	None	0.71	61.74
ExtraTreesClassifier	0.76	0.57	None	0.74	2.94
XGBClassifier	        0.78	0.54	None	0.76	2.51
LGBMClassifier	        0.78	0.53	None	0.75	1.19
RandomForestClassifier	0.76	0.53	None	0.74	3.86
ExtraTreeClassifier	    0.65	0.52	None	0.65	0.06
DecisionTreeClassifier	0.65	0.51	None	0.65	0.24
BaggingClassifier	    0.73	0.50	None	0.71	1.66
KNeighborsClassifier	0.68	0.48	None	0.67	0.34
NearestCentroid	        0.50	0.47	None	0.53	0.06

→オーバーサンプリングは効果なし！

In [ ]:
時系列データを分析に加える

→月、曜日、平日/休日のターゲットエンコーディング値とダミー変数を分析要因に加える
→テキストのTF・IDF値を算出して分析に加える

In [ ]:
→年・月のターゲットエンコーディングはほぼ効かない。次はダミーを含める1
→ほんの少しだけ改善。

In [ ]:
20231218
１、ダミー変数をDropFirst = Trueにする
→効果なし

２、季節をダミー変数にする
３、ダミー変数をレベルエンコーディングに、
　その際ラベルエンコーディングした際の値が低い順にする
３、これまでに作成した予測結果をアンサンブルする

ランダムフォレストの特徴量を可視化する

→年データを含めない・季節データをダミー化しない方が
　精度が向上するのではないか？？

 
 年を含めず精度向上するか、季節データののみで向上するかを検証する
 →年データは含めた方が良い
 
 Seasonをダミー化せずターゲットエンコーディングした方が精度が向上する？
 →逆効果。季節の様なカテゴリー数が少ないデータはターゲットエンコーディングしない方が良い。

 ターゲットエンコーディングしたボロコードの寄与率が異様に高いので、これを排除した方が
 分類精度が上がるのではないか？
 
guards_Helpful	0.000000
curb_loc_tg_enc	0.000000
steward_4orMore	0.000000
guards_Harmful	0.000000
sidewalk_NoDamage	7.125190
user_type_tg_enc	31.364330
borocode_2	68.618100
steward_3or4	71.059918

→これらの寄与率が低いデータが弾かれるようにする



In [ ]:
25

比重付き精度：
0.4054251139399654
マクロf1スコア：
0.397620466909088

In [ ]:
ニューラルネットワークの最適化
・各層のユニット数とドロップアウト率
・バッチ数

result = model_nn.fit(X_tr, y_tr, batch_size=200, epochs=20, validation_data=(X_va, y_va), verbose=1)

In [ ]:
これまでに最高精度が出た時の特徴量を使ってニューラルネットワークで予測を行う
→ニューラルネットでは無理。
精度の高い予測結果でアンサンブリングする

In [ ]:

submit_35_GNB_DateData_Enc+Dammy+Season.csv	0.3589799
 
submit_35_GNB_DateData_Enc+Dammy+Season_NoYear.csv	0.3581829
submit_32_GNB_DateData_Enc+Dammy02.csv	0.3554220
submit_30_GNB_DateData_Enc.csv	0.3534689
submit_32_GNB_DateData_Enc+Dammy.csv	0.3528905

submit_25_GNB_DateData_Enc+Dammy+TMS_TEncOnly.csv	0.3527337
submit_PassiveAggressiveClassifier_01.csv	0.3497088

In [ ]:
最も予測能力が高い
submit_35_GNB_DateData_Enc+Dammy+Season.csvをベースに
他の予測結果を6つ、1:2の割合で加える

ニューヨーク市の当月の平均気温・平均降水量・平均湿度などの天候データを
マッピングして加える。予測精度に寄与するかも検証する

In [ ]:

 	
Jan
Feb
Mar
Apr
May
Jun
Average high in ºF
39	42	50	60	71	79
Average low in ºF
26	29	35	44	55	64
Days with precipitation
11	10	12	11	11	10
Hours of sunshine
154	171	213	237	268	289
Av. precipitation in inch
3.90	2.95	4.06	3.94	4.45	3.50


 	
Jul
Aug
Sep
Oct
Nov
Dec
Average high in ºF
85	83	76	65	54	44
Average low in ºF
70	69	61	50	41	32
Days with precipitation
11	10	8	8	9	10
Hours of sunshine
302	271	235	213	169	155
Av. precipitation in inch
4.53	4.13	3.98	3.39	3.82	3.58

In [ ]:

avg_highf
avg_lowf
days_prcp
inch_prcp
hours_snsn

"avg_highf", "avg_lowf", "days_prcp", "inch_prcp", "hours_snsn"

→ まずは日照量で精度向上するかを検証
→ 逆効果！次はすべての天候データを利用して予測する